In [ ]:
# always import
import sys
from time import time

# numpy & scipy
import numpy as np
import scipy

# sklearn
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import pairwise_distances_argmin, pairwise_distances
from sklearn import neighbors

# Hungarian algorithm
#from munkres import Munkres
from scipy.optimize import linear_sum_assignment

# visuals
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')
%matplotlib inline
from matplotlib import offsetbox
from sklearn.manifold import Isomap, TSNE

# maybe
from numba import jit

In [ ]:
# load MNIST data and normalization
from sklearn.datasets import fetch_openml
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, data_home='mnist/')
y = np.asarray(list(map(int, y)))
X = np.asarray(X.astype(float))
X = scale(X)
n_digits = len(np.unique(y))

print("n_digits: ", n_digits)
print("X.shape: ",X.shape)
print("y.shape: ",y.shape)

In [ ]:
#Visualise input data
plt.figure(figsize=(20,4))
for index, (image, label) in enumerate(zip(X[0:5], y[0:5])):
    plt.subplot(1, 5, index + 1)
    plt.imshow(np.reshape(image, (28,28)), cmap=plt.cm.gray)
    plt.title('Training: %i\n' % label, fontsize = 20)

In [ ]:
X_tsne = X[:500]
y_tsne = y[:500]

tsne = TSNE(n_components=2, random_state=0)
X_2d = tsne.fit_transform(X_tsne)

target_ids = np.unique(y)

plt.figure(figsize=(6, 5))
colors = ['r', 'g', 'b', 'c', 'm', 'y', 'k', 'silver', 'orange', 'purple']
for i, c, label in zip(target_ids, colors, np.unique(y)):
    plt.scatter(X_2d[y_tsne == i, 0], X_2d[y_tsne == i, 1], c=c, label=label)
plt.legend()
plt.show()

In [ ]:
def EuclideanDist(x, C):
    '''
    x: m-dim data
    C: K x m matrix of cluster centroids
    
    return: 
    l2_dist: array of Euclidean distance of x to each of the cluster
    min_clus_ind: row index of C that rep. min. dist between x and K
            
    '''
#     l2_dist = np.sqrt(np.sum(np.square(C - x), axis=1))#K-dim
    l2_dist = np.sqrt(np.diagonal(np.matmul(C-x,(C-x).T)))
    min_clus_ind = np.argmin(l2_dist)
    
    return l2_dist, min_clus_ind

In [ ]:
# Kmeans
def Kmeans(X, K, init='Forgy'):
    tau = 1e-4
    max_iter = 300
    iter = 0
    
    n, m = X.shape
    C = np.zeros((K,m))
    z = np.zeros(n) #clustering label for each xi
    dist_to_clus = np.zeros(n)
    prev_obj = 0
    
    #initialse centroids
    if init == 'Forgy':
        ind = np.random.choice(n,K, replace=False)
        C = X[ind,:]
    elif init == 'PCA':
        pca = PCA(n_components=K)
        pca.fit(X)
        C = pca.components_ #top K eigenectors
    
    while True:
        #assign each xi to closest centroid
        for i in range(n):
            l2_dist, z_i = EuclideanDist(X[i,:], C)
            z[i] = z_i
            dist_to_clus[i] = l2_dist[z_i]

        #update cluster
        for k in range(K):  
            C[k,:] = np.mean(X[(z==k),:], axis=0)
        
        
        #stopping criteria check
        if iter >= 1:
            curr_obj = np.sum(np.square(dist_to_clus))
            if (iter >= max_iter) or (prev_obj - curr_obj <= tau):
                break
            
        iter += 1
        prev_obj =  np.sum(np.square(dist_to_clus))
    
    print('final loss: ', curr_obj)
    print('total Iter: ', iter)
    print('z:', z.shape)
    print('C:', C.shape)
    
    return z, C, curr_obj

In [ ]:
# Evaluate Kmeans on MNIST
# Problem 2(a)-i 
z_pca, C, curr_obj = Kmeans(X,10,'PCA')

In [ ]:
# PCA Evaluation
print('adjusted_rand_score: ', metrics.adjusted_rand_score(y, z_pca))
print('adjusted_mutual_info_score: ', metrics.adjusted_mutual_info_score(y, z_pca))
homogeneity, completeness, v_measure =  metrics.homogeneity_completeness_v_measure(y, z_pca)
print('metrics.homogeneity_score: ', homogeneity)
print('metrics.completeness_score: ', completeness)
print('metrics.v_measure_score: ', v_measure)

In [ ]:
# Problem 2(a)-ii
pca = PCA(n_components=30)
X_proj = pca.fit_transform(X)
run = 10
min_obj = 999999999

print('X_proj: ', X_proj.shape)
print('X: ', X.shape)

n, m_pri = X_proj.shape
z_min = np.zeros(n)
C_min = np.zeros((10,m_pri))

for i in range(run):
    z, C, curr_obj = Kmeans(X_proj, 10)
    
    if curr_obj < min_obj:
        min_obj = curr_obj
        z_min = z
        C_min = C

In [ ]:
# Forgy init with PCA dim reduction evaluation
print('adjusted_rand_score: ', metrics.adjusted_rand_score(y, z_min))
print('adjusted_mutual_info_score: ', metrics.adjusted_mutual_info_score(y, z_min))
homogeneity, completeness, v_measure =  metrics.homogeneity_completeness_v_measure(y, z_min)
print('metrics.homogeneity_score: ', homogeneity)
print('metrics.completeness_score: ', completeness)
print('metrics.v_measure_score: ', v_measure)

In [ ]:
# Hungarian algorithm

# cost matrix, W, construction
def W_gen(K,z,y):
    '''
    K: number of clusters
    z: cluster prediction
    y: true labels
    '''
    n = len(y)
    W = np.zeros((K,K))

    for i,col in enumerate(W): # for each row
        for j, W_ij in enumerate(col): # for each col
            z_l = np.where(z==i) # np.where return indices of matched components
            y_l = np.where(y==j) 

            comm_Elem = np.intersect1d(z_l, y_l)
            W[i][j] = 1.0 - len(comm_Elem)/n

        W[i,:] -= np.min(W[i,:])

    return W
    
def ClusterMap(z, hung_map):
    z_prime = np.zeros(len(z))
    
    for i, mapping in enumerate(hung_map):
        z_prime[np.where(z == i)] = mapping
        
    return z_prime

def precision(pred, y):
    return sum(np.equal(pred, y)) / float(len(y))

In [ ]:
W_PCA = W_gen(10,z_pca,y)
row_pca, col_pca = linear_sum_assignment(W_PCA)
print("col_pca: ", col_pca)

z_pri_pca = ClusterMap(z_pca,col_pca) 
print('PCA precision: ', precision(z_pri_pca, y))

In [ ]:
cf_mat = confusion_matrix(z_pri_pca, y)
fig, ax = plt.subplots(figsize=(16, 10))
sns.heatmap(cf_mat, annot = True, cmap = 'BuGn_r',square=True, fmt='d')
ax.set_ylim(10,0) #to prevent top and bottom from getting cut off
plt.show()
cf_mat

In [ ]:
W_forgy = W_gen(10,z_min,y)
row_forgy, col_forgy = linear_sum_assignment(W_forgy)
print("col_forgy: ", col_forgy)

z_pri_min = ClusterMap(z_min,col_forgy)
print('forgy precision: ', precision(z_pri_min, y))

In [ ]:
cf_mat = confusion_matrix(z_pri_min, y)
fig, ax = plt.subplots(figsize=(16, 10))
sns.heatmap(cf_mat, annot = True, cmap = 'BuGn_r',square=True, fmt='d')
ax.set_ylim(10,0) #to prevent top and bottom from getting cut off
plt.show()
cf_mat

In [ ]:
# for sparse matrix 
# row-wise normalise, modify in-place
def rowWiseNorm(E):
    indptr = E.indptr
    last_idx = indptr[0]
    for row_id, idx in enumerate(indptr[1:]):
        if idx == last_idx: # empty row
            continue
        else:
            row_sum = np.sum(E.data[last_idx:idx])
#             print(type(E.data[last_idx:idx]))
#             print(E.data[last_idx:idx])
#             print(row_sum)
#             np.asarray(E.data[last_idx:idx])

            E.data[last_idx:idx] /= row_sum
            
#             print(E.data[last_idx:idx])
            last_idx = idx

In [ ]:
pca = PCA(n_components=30)
X_pri = pca.fit_transform(X)

# knbrs = neighbors.NearestNeighbors(n_neighbors=500, algorithm='kd_tree', metric='euclidean')
# kgraph = knbrs.fit(X_pri).kneighbors_graph(mode='distance')

#Save kgraph for faster testing
# scipy.sparse.save_npz('./kgraph.npz',kgraph)
# H=scipy.sparse.load_npz('./kgraph.npz')


In [ ]:
# Spectral clustering

def SpectralCluster(X_pri, K, n_neighbors=500, k_eigVec=20):
    
    # csr format
    H = neighbors.NearestNeighbors(n_neighbors=n_neighbors, algorithm='kd_tree', metric='euclidean').fit(X_pri).kneighbors_graph(mode='distance')
#     H=scipy.sparse.load_npz('./kgraph.npz')
    scipy.sparse.save_npz('./kgraph.npz',H)
    print('kgraph saved...\n')
    n = H.shape[0]

    # normalise H
    sigma = H.sum()/H.count_nonzero()  # == np.sum(H.data)
    H.data = H.power(2).data
    H.data = np.exp(-H.data/np.power(sigma,2))

    rowWiseNorm(H)
#     H.data = H.data/H.sum()
    H.setdiag(1.0)

    # smmetrize H
    H = (H.transpose() + H)/2 # H sparse_csr

    # Compute D
    D = np.squeeze(np.asarray(H.sum(axis=1)))
    D = scipy.sparse.diags(D)

    # Compute D^(-1/2)
    D_inv = D.power(-0.5) # sparse_diag

    # Compute L
    L = scipy.sparse.identity(n) - D_inv@H.tocoo()@D_inv
    
    # Top k eigenvectors
#     print('------------eig start-----------------')
    eigVals, eigVecs = scipy.sparse.linalg.eigs(L, k=k_eigVec, which='SM')
#     print('nonzero: ',np.count_nonzero(eigVecs,axis=1))
#     print('------------eig end-----------------')
    
    eigVecs = eigVecs[:,1:].real
    norm_vec = np.sqrt(np.sum(np.power(eigVecs,(2)),axis=1))
    eigVecs_norm = eigVecs/norm_vec.reshape(-1,1)
    
    print('eigVals: ' ,eigVals)
    print('-------------------------\n')
    print('eigVecs: ' ,eigVecs)
    print('-------------------------\n')
    print('eigVecs.shape: ' ,eigVecs.shape)
    print('-------------------------\n')
        
    # run K-means 10 times, return the best
    kmeans = KMeans(init='k-means++', n_clusters=K, n_init=10).fit(eigVecs_norm)
    # n_init: number of times to run KMeans for.
    
    z = kmeans.labels_
    C = kmeans.cluster_centers_
    print('shape(z): ', z.shape)
    print('shape(C): ', C.shape)
    print('shape(eigVecs_norm): ', eigVecs_norm.shape)
    
    return z, C, eigVecs_norm

In [ ]:
z_sptrclus, C, data_speclu= SpectralCluster(X_pri, 10)

In [ ]:
W_sptrclus = W_gen(10,z_sptrclus,y)
row_z_sptrclus, col_z_sptrclus = linear_sum_assignment(W_sptrclus)
print("col_z_sptrclus: ", col_z_sptrclus)
print("row_z_sptrclus: ", row_z_sptrclus)

z_sptrclus_map = ClusterMap(z_sptrclus, col_z_sptrclus)
# print(z_sptrclus_map.shape)

print('spcetral clustering precision: ', precision(z_sptrclus_map, y))

cf_mat = confusion_matrix(z_sptrclus_map, y)
fig, ax = plt.subplots(figsize=(16, 10))
sns.heatmap(cf_mat, annot = True, cmap = 'BuGn_r',square=True, fmt='d')
ax.set_ylim(10,0) #to prevent top and bottom from getting cut off
plt.show()
cf_mat

In [ ]:
z_sptrclus, C, _ = SpectralCluster(X_pri, 10)

In [ ]:
W_sptrclus = W_gen(10,z_sptrclus,y)
row_z_sptrclus, col_z_sptrclus = linear_sum_assignment(W_sptrclus)
print("col_z_sptrclus: ", col_z_sptrclus)
print("row_z_sptrclus: ", row_z_sptrclus)

z_sptrclus_map = ClusterMap(z_sptrclus, col_z_sptrclus)
# print(z_sptrclus_map.shape)

print('spcetral clustering precision: ', precision(z_sptrclus_map, y))

cf_mat = confusion_matrix(z_sptrclus_map, y)
fig, ax = plt.subplots(figsize=(16, 10))
sns.heatmap(cf_mat, annot = True, cmap = 'BuGn_r',square=True, fmt='d')
ax.set_ylim(10,0) #to prevent top and bottom from getting cut off
plt.show()
cf_mat

In [ ]:
# Clustering + KNN
class KNNClassifier():

    def __init__(self,k, ind=None):
        self.kNeighbors = k
        if np.all(ind):
            # for the random choice case
            self.indToTrain = ind
        else:
            # for Kmeans and spectral clustering
            self.indToTrain = np.array([],dtype=np.int)
            
        self.model = None # store cloest pts to each c in C
        self.label = np.array([]) 
        
        # KNN predictions
        self.y_pred = []
        
    def fit(self, C, train, y):
        '''
        for each c in C, find min dist xi and C
            store xi index, xi, and true label of xi
        
        C: shape(100,30) rand & kmeans, shape(100,(k-1)eigVects) spc 
        train: shape(n,30) rand & kmeans, shape(n,(k-1)eigVects) spc
        '''
        self.model = np.zeros((100,train.shape[1]))

        if self.indToTrain.size != 0:
#             print('self.indToTrain:' ,type(self.indToTrain))
            self.model = train[self.indToTrain,:]
            self.label = y[self.indToTrain]
            return
        
        for row_idx , C_row in enumerate(C):
            # l2.shape = (n,1)
            # C[row, :] = C_row
            l2 = np.sqrt(np.sum(np.square(train-C_row.reshape(1,-1)), axis=1))
            least_ind = np.argmin(l2)
            
            self.indToTrain = np.append(self.indToTrain, least_ind)
            self.model[row_idx,:] = train[least_ind,:]
            self.label = np.append(self.label, y[least_ind])
        
    def predict(self, test, y):
        '''
        for each test bar the index ones
           compute dist bet. all points in self.model
           find k smallest dist. get corresponsing labels 
           save max. occurrening labels to self.y_pred
           
        test: same as train but points part of kNN model will be excluded from evaluation
        '''
        for i in range(test.shape[0]):
            if i in self.indToTrain: #skip 100 train samples from D
                continue
            else:
                l2 = np.sqrt(np.sum(np.square(self.model-test[i,:].reshape(1,-1)),axis=1))

                k_smallest_ind = np.argsort(l2)[:self.kNeighbors]
                
                vals, cnts = np.unique(self.label[k_smallest_ind], return_counts=True)
                self.y_pred.append(vals[np.argmax(cnts)])
        
        # mask out self.model from test for calc. precision
        mask = np.ones(test.shape[0], dtype=bool)
        mask[self.indToTrain] = False
        y_masked = y[mask]
        
        return precision(self.y_pred, y_masked)
        

In [ ]:
pca = PCA(n_components=30)
X_pri = pca.fit_transform(X)
n,m = X_pri.shape

print(f'X_pri shape: {X_pri.shape}')

In [ ]:
# Random choice
knbors = [1,3,5]
pred_acc=[]

for k_ind, k in enumerate(knbors):
    print(f'------------------ In {k} ------------------\n')
    ind_rc = np.random.choice(n,100, replace=False)
    C_rc = X_pri[ind_rc, :]

    knn_rand = KNNClassifier(k,ind_rc)
    knn_rand.fit(C_rc, X_pri,y)
    pred_acc.append(knn_rand.predict(X_pri,y))

print('\n')
for i, k in enumerate(pred_acc):
    print(f'precision for random choice with {knbors[i]} neighbors: {k}')

In [ ]:
# Spectral clustering
H = neighbors.NearestNeighbors(n_neighbors=500, algorithm='kd_tree', metric='euclidean').fit(X_pri).kneighbors_graph(mode='distance')
# scipy.sparse.save_npz('./kgraph.npz',H)
# H=scipy.sparse.load_npz('./kgraph.npz')

In [ ]:
z_speclu, C_speclu, data_speclu = SpectralCluster(X_pri, 100, n_neighbors=500, k_eigVec=20)
print(f'data_speclu shape: {data_speclu.shape}')
print(f'C_speclu shape: {C_speclu.shape}')


In [ ]:
knbors = [1,3,5]
pred_acc=[]

for k_ind, k in enumerate(knbors):
    print(f'------------------ In {k} ------------------\n')
    knn_spc = KNNClassifier(k)
    knn_spc.fit(C_speclu, data_speclu, y)
    pred_acc.append(knn_spc.predict(data_speclu, y))

print('\n')
for i, k in enumerate(pred_acc):
    print(f'precision for spectral clustering with {knbors[i]} neighbors: {k}')

In [ ]:
# Kmeans 
# run Kmeans 10 times and pick the best
run = 10
min_obj = 999999999

n, m = X_pri.shape
z_Kmeans = np.zeros(n)
C_Kmeans = np.zeros((100,m))

for i in range(run):
    print(f'--------------run: {i}--------------------')
    z, C, curr_obj = Kmeans(X_pri, 100, 'Forgy')
    
    if curr_obj < min_obj:
        min_obj = curr_obj
        z_Kmeans = z
        C_Kmeans = C
        
print(f'C_Kmeans shape: {C_Kmeans.shape}')

In [ ]:
# Kmeans 
knbors = [1,3,5]
pred_acc=[]

for k_ind, k in enumerate(knbors):
    print(f'------------------ In {k} ------------------\n')
    knn_Kmeans = KNNClassifier(k)
    knn_Kmeans.fit(C_Kmeans, X_pri, y)
    pred_acc.append(knn_Kmeans.predict(X_pri, y))

print('\n')
for i, k in enumerate(pred_acc):
    print(f'precision for Kmeans with {knbors[i]} neighbors: {k}')